In [1]:
import os 
import napari
from tifffile import imread, imwrite
from skimage.measure import regionprops_table
from skimage.segmentation import watershed
from scipy import ndimage as ndi
from skimage.feature import peak_local_max
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from cellpose import models

In [2]:
# set napari to block execution
from napari.settings import get_settings

settings = get_settings()

## Loading data

In [3]:
data_dir = r'D:\data_analysis\2023_Andrew\data'

In [4]:
model = models.Cellpose(model_type='cyto2',gpu=True)

In [5]:
def correct_mitochondria(mito_mask,im):

    # create a binary mask of previously detected mitochondria
    mito_binary = mito_mask > 0

    # create fake image to help with segmentation
    im_to_seg = 0.3*im + 150*mito_binary

    # segment mitochondria with cellpose
    corr_mask, _, _, _ = model.eval(im_to_seg, diameter = 75)

    # keep only objects that were detected originally
    corr_mask = corr_mask * mito_binary

    # if there are originally detected objects that don't have
    # any overlap with new objects keep them as well
    properties = ['label','image','mean_intensity','bbox']
    df = pd.DataFrame(regionprops_table(mito_mask,intensity_image=(corr_mask>0),properties=properties))
    df = df.loc[df.mean_intensity<0.1,:]

    i = np.max(corr_mask) + 1
    for ind,m in df.iterrows():

        corr_mask[m['bbox-0']:m['bbox-2'],m['bbox-1']:m['bbox-3']] = corr_mask[m['bbox-0']:m['bbox-2'],m['bbox-1']:m['bbox-3']] + m.image*i
        i += 1

    return corr_mask

In [6]:
properties = ['label','area','axis_major_length','axis_minor_length','eccentricity','centroid']

sub_dir_list = [x for x in os.listdir(data_dir) if not('Store' in x)]

df_list =[]

settings.application.ipy_interactive = False
for sub_dir in sub_dir_list:

    # list the directory
    sub_sub_dir_list = [x for x in os.listdir(os.path.join(data_dir,sub_dir)) if '300' in x]

    for sub_sub_dir in sub_sub_dir_list:

        # list the the directory
        sub_sub_path = os.path.join(data_dir,sub_dir,sub_sub_dir)
        sub_sub_sub_dir_list = [x for x in os.listdir(sub_sub_path) if os.path.isdir(os.path.join(sub_sub_path,x))]

        for sub_sub_sub_dir in sub_sub_sub_dir_list:

            print(os.path.join(data_dir,sub_dir,sub_sub_dir,sub_sub_sub_dir))

            # get list of all the images in this directory
            im_list = os.listdir(os.path.join(data_dir,sub_dir,sub_sub_dir,sub_sub_sub_dir))

            # find the intensity value image
            image_em_name = [x for x in im_list if (('tif' in x) and not(('er' in x) or ('ER' in x) or ('frags' in x) or ('cells.' in x) or ('mit' in x)))]
            
            # check that there is only intensity image in this directory
            if (len(image_em_name) == 1):
                image_em_name = image_em_name[0]
                print('Image found.')
            else:
                print('Error in finding EM image.')

            
            # read in intensity image
            image_em_path = os.path.join(sub_sub_path,sub_sub_sub_dir,image_em_name)
            image_em = imread(image_em_path)

            # find and read in mitochondria mask
            labels_mitos_name = [x for x in im_list if (('.tif' in x) and ('mitos' in x))][0]
            labels_mitos_path = os.path.join(sub_sub_path,sub_sub_sub_dir,labels_mitos_name)
            labels_mitos = imread(labels_mitos_path)

            # recalculate mitos labels
            labels_mitos_corr = correct_mitochondria(labels_mitos,image_em)

            # visualize
            viewer = napari.Viewer()
            viewer.add_image(image_em)
            viewer.add_labels(labels_mitos,name='mitos',visible=False)
            viewer.add_labels(((labels_mitos_corr>0) != (labels_mitos>0))*2,name='diff')
            viewer.add_labels(labels_mitos_corr,name='mitos_corr')
            napari.run()

            # save corrected mask
            mask_corr = viewer.layers['mitos_corr'].data
            labels_mitos_corr = labels_mitos_name.replace('mitos','mit_corr')
            labels_corr_path = os.path.join(sub_sub_path,sub_sub_sub_dir,labels_mitos_corr)
            imwrite(labels_corr_path,mask_corr)



settings.application.ipy_interactive = True       


D:\data_analysis\2023_Andrew\data\DCA treated pmel t cells day 8\300k pic 1\DCA 300k pic1 collected
Image found.
D:\data_analysis\2023_Andrew\data\DCA treated pmel t cells day 8\300k pic 10\300k_dca_pic10_collected
Image found.
D:\data_analysis\2023_Andrew\data\DCA treated pmel t cells day 8\300k pic 11\300k_dca_pic11_collected
Image found.
D:\data_analysis\2023_Andrew\data\DCA treated pmel t cells day 8\300k pic 12\300k_dca_pic12_collected
Image found.
D:\data_analysis\2023_Andrew\data\DCA treated pmel t cells day 8\300k pic 13\300k_DCA_pic13_collected
Image found.
D:\data_analysis\2023_Andrew\data\DCA treated pmel t cells day 8\300k pic 14\300k_dca_pic14_collected
Image found.
D:\data_analysis\2023_Andrew\data\DCA treated pmel t cells day 8\300k pic 14\300k_DCA_pic1_collected
Image found.
D:\data_analysis\2023_Andrew\data\DCA treated pmel t cells day 8\300k pic 2\300k_dca_pic2_collected
Image found.
D:\data_analysis\2023_Andrew\data\DCA treated pmel t cells day 8\300k pic 3\300k_dca_

## Just visualization

In [6]:
sub_dir_list = [x for x in os.listdir(data_dir) if not('Store' in x)]

df_list =[]

settings.application.ipy_interactive = False
for sub_dir in sub_dir_list:

    # list the directory
    sub_sub_dir_list = [x for x in os.listdir(os.path.join(data_dir,sub_dir)) if '300' in x]

    for sub_sub_dir in sub_sub_dir_list:

        # list the the directory
        sub_sub_path = os.path.join(data_dir,sub_dir,sub_sub_dir)
        sub_sub_sub_dir_list = [x for x in os.listdir(sub_sub_path) if os.path.isdir(os.path.join(sub_sub_path,x))]

        for sub_sub_sub_dir in sub_sub_sub_dir_list:

            print(os.path.join(data_dir,sub_dir,sub_sub_dir,sub_sub_sub_dir))

            # get list of all the images in this directory
            im_list = os.listdir(os.path.join(data_dir,sub_dir,sub_sub_dir,sub_sub_sub_dir))

            # find the intensity value image
            image_em_name = [x for x in im_list if (('tif' in x) and not(('er' in x) or ('ER' in x) or ('frags' in x) or ('cells.' in x) or ('mit' in x)))]
            
            # check that there is only intensity image in this directory
            if (len(image_em_name) == 1):
                image_em_name = image_em_name[0]
                print('Image found.')
            else:
                print('Error in finding EM image.')

            
            # read in intensity image
            image_em_path = os.path.join(sub_sub_path,sub_sub_sub_dir,image_em_name)
            image_em = imread(image_em_path)

            # find and read in mitochondria mask
            labels_mitos_name = [x for x in im_list if (('.tif' in x) and ('corr' in x))][0]
            labels_mitos_path = os.path.join(sub_sub_path,sub_sub_sub_dir,labels_mitos_name)
            labels_mitos = imread(labels_mitos_path)


            # visualize
            viewer = napari.Viewer()
            viewer.add_image(image_em)
            viewer.add_labels(labels_mitos,name='mitos')
            napari.run()


settings.application.ipy_interactive = True       


D:\data_analysis\2023_Andrew\data\DCA treated pmel t cells day 8\300k pic 1\DCA 300k pic1 collected
Image found.
D:\data_analysis\2023_Andrew\data\DCA treated pmel t cells day 8\300k pic 10\300k_dca_pic10_collected
Image found.
D:\data_analysis\2023_Andrew\data\DCA treated pmel t cells day 8\300k pic 11\300k_dca_pic11_collected
Image found.
